# Strategy

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
import backtesting as bck
from plotly import tools
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot, iplot_mpl
import datetime as dt
from pandas_datareader.yahoo.fx import YahooFXReader
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Backtest
from scipy.signal import argrelextrema

## Strategy

### Define Functions

In [2]:
# define moving average
def SMA(values, n):
    return pd.Series(values).rolling(n).mean() 


In [3]:
# define upper Bollinger band
def STD_upper(values, n, n_std): # definire funzione
    return pd.Series(values).rolling(n).mean() + n_std * pd.Series(values).rolling(n).std() 

In [4]:
# define lower Bollinger band
def STD_lower(values, n, n_std): 
    return pd.Series(values).rolling(n).mean() - n_std * pd.Series(values).rolling(n).std() 

In [5]:
# define support
def support(values, n):
    df = pd.DataFrame(values, columns=['Close']) # used dataframe to store values because of the use of argrelextrema
    df['support'] = df.iloc[argrelextrema(df['Close'].values, np.less_equal, order=n)[0]]['Close']
    df['support'] = df['support'].fillna(method='ffill') # argrlextrema returns NaN values, so fill them with the previous value
    return df['support'].values

In [6]:
# define resistance
def resistance(values, n):
    df = pd.DataFrame(values, columns=['Close'])
    df['resistance'] = df.iloc[argrelextrema(df['Close'].values, np.greater_equal, order=n)[0]]['Close']
    df['resistance'] = df['resistance'].fillna(method='ffill')
    return df['resistance'].values

#### Bollinger (close position at std)

In [7]:
# define Bollinger-SupportResistance strategy
class BollingerSrMean(TrailingStrategy): # child class of TrailingStrategy (to add trailing stop loss)

    # parameters are already optimized
    horizon = 30 # 20
    n_std_upper = 3 # 2.4 or full train 2.6
    n_std_lower = 2.8 # 2.2 or full train 2.6
    stop_loss = 7
    n = 10
    # or training on last two years before the test set:
    # horizon=30,stop_loss=7,n_std_upper=2.9999999999999996,n_std_lower=2.8,n=10)

    def init(self):

      super().init() # call parent class (TrailingStrategy)
      self.sma = self.I(SMA, self.data.Close, self.horizon) # simple moving average
      self.upper = self.I(STD_upper, self.data.Close, self.horizon, self.n_std_upper) # upper band
      self.lower = self.I(STD_lower, self.data.Close, self.horizon, self.n_std_lower) # lower band
      self.support = self.I(support, self.data.Close, self.n) # support
      self.resistance = self.I(resistance, self.data.Close, self.n) # resistance   
      self.set_trailing_sl(self.stop_loss) # stop loss


    def next(self): 
      super().next() # call parent class (TrailingStrategy)
      
      if crossover(self.lower, self.data.Close) and self.data.Close[-1] >= self.support[-1]: # if price close falls below the lower band and the close price is above the support...
        self.position.close() # close position
        self.buy() # buy

      elif crossover(self.data.Close, self.upper) and self.data.Close[-1] <= self.resistance[-1]: # if price close rises above the upper band and the close price is below the resistance...
        self.position.close() # close position
        self.sell() # sell



### Import Data

In [8]:
# load data and show first 5 rows
df = pd.read_csv('usdzar_30m.txt')
df.head()

,Date,Time,Open,High,Low,Close,Up,Down
0,10/21/2002,08:30,10.3397,10.3497,10.3147,10.3167,0,0
1,10/21/2002,09:00,10.3397,10.3397,10.2867,10.3197,0,0
2,10/21/2002,09:30,10.2997,10.3452,10.2997,10.3197,0,0
3,10/21/2002,10:00,10.3402,10.3535,10.3197,10.3367,0,0
4,10/21/2002,10:30,10.3297,10.3552,10.3197,10.3432,0,0


In [9]:
# check data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248736 entries, 0 to 248735
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Date    248736 non-null  object 
 1   Time    248736 non-null  object 
 2   Open    248736 non-null  float64
 3   High    248736 non-null  float64
 4   Low     248736 non-null  float64
 5   Close   248736 non-null  float64
 6   Up      248736 non-null  int64  
 7   Down    248736 non-null  int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 15.2+ MB


In [10]:
# create string containing date and time
df['Date'] = df['Date'] + ' ' + df['Time']

In [11]:
# convert string to datetime (call it "Time", seen in backtesting library docs but Date should work too)
df['Time'] = pd.to_datetime(df['Date'])

In [12]:
# drop unnecessary columns
df = df.drop(['Up', 'Down', 'Date'], axis = 1)

In [13]:
# show first 5 rows
df.head()

,Time,Open,High,Low,Close
0,2002-10-21 08:30:00,10.3397,10.3497,10.3147,10.3167
1,2002-10-21 09:00:00,10.3397,10.3397,10.2867,10.3197
2,2002-10-21 09:30:00,10.2997,10.3452,10.2997,10.3197
3,2002-10-21 10:00:00,10.3402,10.3535,10.3197,10.3367
4,2002-10-21 10:30:00,10.3297,10.3552,10.3197,10.3432


In [14]:
# set index to "Time"
df = df.set_index('Time', drop = True)

In [15]:
# split into train and test
date_split = dt.datetime(2023, 1, 1, 0, 0, 0) # data per separare
train = df.loc[:date_split, :] # selezionare dati precedenti
test = df.loc[date_split:, :] # selezionare dati successivi

In [16]:
# show lengths of train and test
print(f'len train: {len(train)}, len test: {len(test)}')

len train: 233421, len test: 15315


In [17]:
# show first 5 rows of train
train.head()

,Open,High,Low,Close
Time,,,,
2002-10-21 08:30:00,10.3397,10.3497,10.3147,10.3167
2002-10-21 09:00:00,10.3397,10.3397,10.2867,10.3197
2002-10-21 09:30:00,10.2997,10.3452,10.2997,10.3197
2002-10-21 10:00:00,10.3402,10.3535,10.3197,10.3367
2002-10-21 10:30:00,10.3297,10.3552,10.3197,10.3432


In [18]:
# run backtest on full train data
bt = Backtest(train[-len(test)*2:], BollingerSrMean, cash=10_000, commission=.002) # commisioni a 0.002 (forse troppo basse)
stats = bt.run()
stats

Start                     2020-07-09 04:00:00
End                       2022-12-30 22:59:00
Duration                    904 days 18:59:00
Exposure Time [%]                    53.95364
Equity Final [$]                 20479.793408
Equity Peak [$]                  20709.772241
Return [%]                         104.797934
Buy & Hold Return [%]                0.379785
Return (Ann.) [%]                   26.832048
Volatility (Ann.) [%]               13.989175
Sharpe Ratio                         1.918058
Sortino Ratio                         4.47233
Calmar Ratio                         3.043296
Max. Drawdown [%]                   -8.816773
Avg. Drawdown [%]                   -0.507892
Max. Drawdown Duration      154 days 13:30:00
Avg. Drawdown Duration        2 days 09:57:00
# Trades                                  209
Win Rate [%]                        48.803828
Best Trade [%]                       4.536003
Worst Trade [%]                     -1.914063
Avg. Trade [%]                    

In [32]:
# plot results
# !!!ATTENTION!!!: because of high number of data points the plot may be slow to load and might crash/freeze the browser
bt.plot(resample=False) # resample=False to show all data points and avoid errors (otherwise backtesting automatically resamples data, failing and returning an error)

c:\Users\lepol\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\lepol\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\lepol\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\lepol\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

c:\Users\lepol\AppData\Local\Programs\Python\Pytho

GridPlot(id='p1311', ...)

In [33]:
# optimization, takes around an hour to run...LEAVE IT COMMENTED!!!

stats = bt.optimize(horizon = [10, 15, 20, 25, 30],
                    stop_loss = range(1, 8),
                    n_std_upper = list(np.arange(1, 4, 0.2)),
                    n_std_lower = list(np.arange(1, 4, 0.2)),
                    n = [10, 15, 20, 25, 30],                
                    maximize='Equity Final [$]' # risultato da massimizzare (per ora equity final, aspetto indicazioni)
)
stats

c:\Users\lepol\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\backtesting.py:1488: UserWarning:

Searching for best of 39375 configurations.



  0%|          | 0/132 [00:00<?, ?it/s]

c:\Users\lepol\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning:

divide by zero encountered in scalar divide

c:\Users\lepol\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning:

divide by zero encountered in scalar divide

c:\Users\lepol\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning:

divide by zero encountered in scalar divide

c:\Users\lepol\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning:

divide by zero encountered in scalar divide

c:\Users\lepol\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning:

divide by zero encountered in scalar divide

c:\Users\lepol\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning:

divide by zero encountered in scalar divide

c:\Users\lepol\AppData\Local

Start                     2020-07-09 04:00:00
End                       2022-12-30 22:59:00
Duration                    904 days 18:59:00
Exposure Time [%]                    53.95364
Equity Final [$]                 20479.793408
Equity Peak [$]                  20709.772241
Return [%]                         104.797934
Buy & Hold Return [%]                0.379785
Return (Ann.) [%]                   26.832048
Volatility (Ann.) [%]               13.989175
Sharpe Ratio                         1.918058
Sortino Ratio                         4.47233
Calmar Ratio                         3.043296
Max. Drawdown [%]                   -8.816773
Avg. Drawdown [%]                   -0.507892
Max. Drawdown Duration      154 days 13:30:00
Avg. Drawdown Duration        2 days 09:57:00
# Trades                                  209
Win Rate [%]                        48.803828
Best Trade [%]                       4.536003
Worst Trade [%]                     -1.914063
Avg. Trade [%]                    

In [34]:
stats._strategy

<Strategy BollingerSrMean(horizon=30,stop_loss=7,n_std_upper=2.9999999999999996,n_std_lower=2.8,n=10)>

In [19]:
# validate on test data
bt = Backtest(test, BollingerSrMean, cash=10_000, commission=.002) 
stats = bt.run()
stats

Start                     2023-01-02 00:30:00
End                       2024-03-22 22:59:00
Duration                    445 days 22:29:00
Exposure Time [%]                   57.486125
Equity Final [$]                 12474.732679
Equity Peak [$]                  12973.173014
Return [%]                          24.747327
Buy & Hold Return [%]               11.886498
Return (Ann.) [%]                   15.660496
Volatility (Ann.) [%]               11.666351
Sharpe Ratio                         1.342365
Sortino Ratio                        2.580984
Calmar Ratio                         1.571929
Max. Drawdown [%]                   -9.962599
Avg. Drawdown [%]                   -0.583845
Max. Drawdown Duration      121 days 08:59:00
Avg. Drawdown Duration        3 days 11:33:00
# Trades                                  112
Win Rate [%]                        46.428571
Best Trade [%]                        4.68053
Worst Trade [%]                     -1.642924
Avg. Trade [%]                    

In [20]:
# plot test results
bt.plot(resample=False) # resample=False to show all data points and avoid errors (otherwise backtesting automatically resamples data, failing and returning an error)

c:\Users\lepol\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\lepol\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'

c:\Users\lepol\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:250: BokehDeprecationWarning:

Passing lists of formats for DatetimeTickFormatter scales was deprecated in Bokeh 3.0. Configure a single string format for each scale

c:\Users\lepol\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:250: UserWarning:

DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'

c:\Users\lepol\AppData\Local\Programs\Python\Pytho

GridPlot(id='p1311', ...)